In [ ]:
from ipywidgets import Layout, Button, Box, HBox, Text, Textarea, Dropdown, Output

In [ ]:
def on_value_change(change):
    out.clear_output()

In [ ]:
name = Text(
    value='',
    placeholder='name of the Docker image',
    description='Name:',
    disabled=False
)
name.observe(on_value_change, names='value')

In [ ]:
ubuntu = Dropdown(
    options=['16.04', '18.04'],
    value='16.04',
    description='Ubuntu:',
    disabled=False,
)
ubuntu.observe(on_value_change, names='value')

In [ ]:
opengl = Dropdown(
    options=['none', 'runtime', 'devel'],
    value='none',
    description='OpenGL:',
    disabled=False,
)
opengl.observe(on_value_change, names='value')

In [ ]:
cuda = Dropdown(
    options=['none', '8.0-runtime', '8.0-devel', '9.0-runtime', '9.0-devel', 
             '9.2-runtime', '9.2-devel', '10.0-runtime', '10.0-devel'],
    value='none',
    description='CUDA:',
    disabled=False,
)
cuda.observe(on_value_change, names='value')

In [ ]:
cudnn = Dropdown(
    options=['none', '5-runtime', '5-devel', '6-runtime', '6-devel', 
             '7-runtime', '7-devel'],
    value='none',
    description='cuDNN:',
    disabled=False,
)
cudnn.observe(on_value_change, names='value')

In [ ]:
ros = Dropdown(
    options=['none', 'kinetic-ros-core', 'kinetic-ros-base', 'kinetic-robot', 'kinetic-perception',
             'kinetic-desktop', 'kinetic-desktop-full', 'melodic-ros-core', 'melodic-ros-base', 
             'melodic-robot', 'melodic-perception', 'melodic-desktop', 'melodic-desktop-full'],
    value='none',
    description='ROS:',
    disabled=False,
)
ros.observe(on_value_change, names='value')

In [ ]:
build = Dropdown(
    options=['none', 'cmake', 'catkin_make', 'catkin_build'],
    value='none',
    description='Build:',
    disabled=False,
)
build.observe(on_value_change, names='value')

In [ ]:
apt = Textarea(
    value='',
    placeholder='List of APT packages\n(one package per line)',
    description='APT:',
    disabled=False
)
apt.observe(on_value_change, names='value')

In [ ]:
pip2 = Textarea(
    value='',
    placeholder='List of Python 2 packages\n(one package per line)',
    description='pip2:',
    disabled=False
)
pip2.observe(on_value_change, names='value')

In [ ]:
pip3 = Textarea(
    value='',
    placeholder='List of Python 3 packages\n(one package per line)',
    description='pip3:',
    disabled=False
)
pip3.observe(on_value_change, names='value')

In [ ]:
matlab = Text(
    value='',
    placeholder='path in the local host',
    description='MATLAB:',
    disabled=False
)
matlab.observe(on_value_change, names='value')

In [ ]:
import os, yaml
from roslab_create import write_docker_file
from IPython.core.display import display, HTML
from IPython.display import clear_output

def generate_dockerfile(b):
    out.clear_output()
    with out:
        if name.value is '':
            print('Error: name is empty!')
        else:
            print("Generating Dockerfile...")
            base = {'ubuntu': ubuntu.value}
            if not opengl.value is 'none':
                base['opengl'] = opengl.value
            if not cuda.value is 'none':
                base['cuda'] = cuda.value
            if not cudnn.value is 'none':
                base['cudnn'] = cudnn.value
            if not ros.value is 'none':
                base['ros'] = ros.value
            data = {'name': name.value, 'base': base}
            if not build.value is 'none':
                data['build'] = build.value
            if not apt.value is '':
                data['apt'] = apt.value.splitlines()
            if not pip2.value is '':
                data['pip'] = pip2.value.splitlines()
            if not pip3.value is '':
                data['pip3'] = pip3.value.splitlines()
            if not matlab.value is '':
                data['matlab'] = {}
                data['matlab']['host_path'] = matlab.value()
            with open('roslab.yaml', 'w') as outfile:
                yaml.dump(data, outfile, default_flow_style=False)
            write_docker_file(data)
            print('Done!')
            os.rename('roslab.dockerfile', 'Dockerfile')
            display(HTML('<p>It can be downloaded <a href="Dockerfile" target="_blank">from here</a>.</p>'))

In [ ]:
generate = Button(
    description='Proceed',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Generate Dockerfile'
)
generate.on_click(generate_dockerfile)

In [ ]:
out = Output()

In [ ]:
form_item_layout = Layout(
    display='flex',
    flex_flow='row',
    justify_content='space-between'
)

left_items = [name, ubuntu, opengl, cuda, cudnn, ros, build, generate, out]
right_items = [apt, pip2, pip3, matlab]

left_form = Box(left_items, layout=Layout(
    display='flex',
    flex_flow='column',
    #border='solid 1px',
    align_items='stretch',
    width='33%'
))
right_form = Box(right_items, layout=Layout(
    display='flex',
    flex_flow='column',
    #border='solid 1px',
    align_items='stretch',
    width='33%'
))
out.clear_output()
HBox([left_form, right_form])